## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv", encoding='utf-8')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < 200].index.tolist()
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")
    
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
cutoff = 1
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 200
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True)


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns='IS_SUCCESSFUL')
# Split the preprocessed data into a training and testing dataset
y = application_df['IS_SUCCESSFUL']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled
X_test_scaled

array([[ 0.01350021, -0.03067262, -0.12527962, ..., -0.06459105,
        -0.07135527, -0.02832705],
       [ 0.01350021, -0.03115182, -0.12527962, ..., -0.06459105,
        -0.07135527, -0.02832705],
       [ 0.01350021, -0.03115182, -0.12527962, ..., -0.06459105,
        -0.07135527, -0.02832705],
       ...,
       [ 0.01350021, -0.03115182,  7.98214399, ..., -0.06459105,
        -0.07135527, -0.02832705],
       [ 0.01350021, -0.03115182, -0.12527962, ..., -0.06459105,
        -0.07135527, -0.02832705],
       [ 0.01350021, -0.03115182, -0.12527962, ..., -0.06459105,
        -0.07135527, -0.02832705]])

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

C:\Users\amid1\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           4,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,313 (52.00 KB)

 Trainable params: 13,313 (52.00 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(loss='binary_crossentropy', 
            optimizer='adam', 
            metrics=['accuracy'])

In [14]:
# Train the model
history = nn.fit(X_train_scaled, 
                  y_train, 
                  epochs=100, 
                  batch_size=32, 
                  validation_split=0.2)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7051 - loss: 0.5908 - val_accuracy: 0.7405 - val_loss: 0.5526
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 796us/step - accuracy: 0.7255 - loss: 0.5608 - val_accuracy: 0.7400 - val_loss: 0.5499
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 797us/step - accuracy: 0.7318 - loss: 0.5505 - val_accuracy: 0.7376 - val_loss: 0.5478
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 787us/step - accuracy: 0.7291 - loss: 0.5531 - val_accuracy: 0.7405 - val_loss: 0.5432
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step - accuracy: 0.7312 - loss: 0.5500 - val_accuracy: 0.7402 - val_loss: 0.5419
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.7333 - loss: 0.5463 - val_accuracy: 0.7389 - val_loss: 0.5454
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 835us/step - accuracy: 0.7322 - loss: 0.5469 - val_accuracy: 0.7376 - val_loss: 0.5498
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 839us/step - accuracy: 0.7274 - loss: 0.5

686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 944us/step - accuracy: 0.7393 - loss: 0.5373 - val_accuracy: 0.7389 - val_loss: 0.5419
Epoch 51/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7390 - loss: 0.5354 - val_accuracy: 0.7378 - val_loss: 0.5499
Epoch 52/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 810us/step - accuracy: 0.7380 - loss: 0.5354 - val_accuracy: 0.7385 - val_loss: 0.5445
Epoch 53/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 782us/step - accuracy: 0.7353 - loss: 0.5357 - val_accuracy: 0.7413 - val_loss: 0.5458
Epoch 54/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - accuracy: 0.7397 - loss: 0.5338 - val_accuracy: 0.7411 - val_loss: 0.5422
Epoch 55/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - accuracy: 0.7395 - loss: 0.5335 - val_accuracy: 0.7402 - val_loss: 0.5455
Epoch 56/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 835us/step - accuracy: 0.7403 - loss: 0.5344 - val_accuracy: 0.7380 - val_loss: 0.5439
Epoch 57/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 834us/step - accuracy: 0.7382 - loss: 0.5348 -

686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 799us/step - accuracy: 0.7429 - loss: 0.5296 - val_accuracy: 0.7392 - val_loss: 0.5507
Epoch 100/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 793us/step - accuracy: 0.7430 - loss: 0.5270 - val_accuracy: 0.7398 - val_loss: 0.5523


In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 515us/step - accuracy: 0.7268 - loss: 0.5697
Loss: 0.5696521997451782, Accuracy: 0.7268221378326416


In [16]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.keras')